# Part 3: Training with Amazon SageMaker Built-in XGBoost 

## Contents
1. [Introduction](#Introduction)
2. [Prerequisites](#Prerequisites)
3. [Train a model using SageMaker XGBoost](#Train-a-model-using-SageMaker-XGBoost)
4. [Set up hosting for the model](#Set-up-hosting-for-the-model)
5. [Perform Inference](#Perform-Inference)
6. [Stop/Close the Endpoint](#Stop-/-Close-the-Endpoint)

## Introduction

SageMaker의 관리형, 분산 학습 프레임워크를 이용하여 학습할 수 있도록 built-in xgboost 모델을 시작해 보겠습니다.

## Prerequisites
### Import libraries

Notebook을 진행하기 위해 여러 라이브러리를 다운로드 합니다.

In [1]:
import os                                         # For manipulating filepath names  
import sys                                        # For writing outputs to notebook
import math                                       # For ceiling function
import json                                       # For parsing hosting outputs

import numpy as np                                # For matrix operations and numerical processing
import pandas as pd                               # For munging tabular data
from time import gmtime, strftime                 # For labeling SageMaker models, endpoints, etc.

import sagemaker                                  # Amazon SageMaker's Python SDK provides many helper functions
from sagemaker import get_execution_role          # Define IAM role
import boto3

### Read dataset

위의 셀에서 설정한 S3 bucket 위치에서 데이터를 불러옵니다.

In [2]:
df = pd.read_csv('./banking_fraud_final_dataset.csv')

df.shape

(3000, 19)

In [17]:
df.head()

,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,0,1,2,3,4
0,CASH_OUT,140421.18,C1667570766,16004.0,0.00,C2102410298,0.00,140421.18,0,0,1,0,0,0,0.014042,0.000279,0.000000,0.000000,0.001334
1,CASH_OUT,216666.53,C1495945377,50398.0,0.00,C814408370,10119297.16,10335963.70,0,0,1,0,0,0,0.021667,0.000879,0.000000,0.096030,0.098211
2,CASH_OUT,234636.20,C269129885,74262.0,0.00,C1389815469,166046.48,400682.68,0,0,1,0,0,0,0.023464,0.001296,0.000000,0.001576,0.003807
3,CASH_IN,52816.29,C129678616,117751.0,170567.29,C842027837,0.00,0.00,0,1,0,0,0,0,0.005282,0.002054,0.003605,0.000000,0.000000
4,CASH_OUT,63871.25,C1282823885,6012.0,0.00,C1236511065,456488.36,520359.60,0,0,1,0,0,0,0.006387,0.000105,0.000000,0.004332,0.004944


### Split dataset

데이터를 train | validation | test 데이터셋으로 분리한 후 모델을 훈련합니다.

In [3]:
# 훈련에 필요없는 column을 삭제합니다
columns_to_drop = ['type', 'isFraud', 'nameOrig', 'nameDest']
target = 'isFraud'

In [4]:
# 데이터를 train, validation, test 각각 7:2:1의 비율로 나눕니다
train_data, validation_data, test_data = np.split(df.sample(frac=1, random_state=2021), [int(0.7 * len(df)), int(0.9 * len(df))])

Amazon SageMaker의 XGBoost 컨테이너는 libSVM 또는 CSV 포맷의 데이터를 사용합니다. 본 예제에서는 CSV를 이용합니다. CSV파일에서 첫번째 컬럼을 타겟변수 값으로 지정해야 하고, 헤더를 포함하고 있지 않아야 합니다. 본 예제에서는 데이터를 train | validation | test 데이터셋으로 분리한 후 작업합니다.

In [5]:
# 각 데이터셋을 노트북 로컬 환경에 저장합니다
pd.concat([train_data[target], train_data.drop(columns_to_drop, axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data[target], validation_data.drop(columns_to_drop, axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)
pd.concat([test_data[target], test_data.drop(columns_to_drop, axis=1)], axis=1).to_csv('test.csv', index=False, header=False)
pd.concat([test_data.drop(columns_to_drop, axis=1)], axis=1).to_csv('test_features.csv', index=False, header=False)

SageMaker의 관리형 학습환경에서 이 데이터에 접근할 수 있도록 파일을 S3에 업로드 합니다.

In [6]:
%store -r bucket
%store -r prefix

In [7]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test_features.csv')).upload_file('test.csv')

## Train a model using SageMaker XGBoost

SageMaker 의 ECR 컨테이너를 통해 해당되는 built-in 알고리즘을 사용할 수 있습니다.

In [8]:
from sagemaker import image_uris

container = sagemaker.image_uris.retrieve(framework = 'xgboost', 
                                          region = boto3.Session().region_name, 
                                          version = 'latest')

CSV 파일 포맷을 사용하므로 S3의 파일 위치를 알려주는 s3_input 오브젝트를 생성하고 콘텐츠 타입을 CSV로 지정합니다.

In [9]:
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

다음으로 파라미터를 지정하여 esitmator를 생성합니다.

- xgboost 알고리즘 컨테이너 사용
- 사용할 IAM 역할(role)
- 학습용 인스턴스 타입과 수량 ('local_cpu'를 사용해 해당 노트북의 인스턴스 내에서 학습을 진행할 수도 있습니다.)
- 출력 데이터를 위한 S3위치
- 알고리즘 하이퍼파라미터

이제 다음 파라미터를 이용하여 .fit() 명령을 실행합니다.
- 학습용(train)/검증용(validation) 데이터가 있는 S3 위치

본 예제는 학습과 검증 데이터셋을 모두 사용하므로 두 채널을 모두 지정합니다. Trainin job을 수행하기 위해서 학습용 서버가 생성되는데에 5분정도 소요됩니다.

데이터 학습을 수행할 시에 발생하는 과금은, EC2 인스턴스의 생성 시간이 제외된 데이터를 학습하는 시간만 요금이 부과됩니다. Log의 마지막 부분에 표시되는 Training seconds와 Billable seconds를 참고하셔서 과금이 발생한 시간을 참고하실 수 있습니다.
- Training seconds: Training job을 실행한 실제 컴퓨팅 학습 시간
- Billable seconds: Spot 할인이 적용된 후 청구되는 시간

---
### 아래 값을 변경해주세요.

Training estimator에 사용되는 instance count와 instance type을 설정합니다. 

본 예제 데이터는 크기가 작기에 instance count는 1로 고정합니다.

Instance type에는 표준 인스턴스 타입 중 하나인 **ml.m5.xlarge**를 사용합니다. 더 많은 인스턴스 타입은 [여기](https://aws.amazon.com/ko/sagemaker/pricing/)에서 확인할 수 있습니다.

추가적으로 Hyperparameters의 값을 조정하며 결과를 비교합니다.

---

In [10]:
sess = sagemaker.Session()
role = get_execution_role()

job_name='banking-fraud-'+strftime("%Y-%m-%d-%H-%M-%S", gmtime())

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='PUT INSTANCE TYPE',           # ml.m5.xlarge
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess,
                                    base_job_name=job_name)

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=5)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation})

2022-04-10 07:07:46 Starting - Starting the training job...
2022-04-10 07:08:12 Starting - Preparing the instances for trainingProfilerReport-1649574466: InProgress
.........
2022-04-10 07:09:38 Downloading - Downloading input data
2022-04-10 07:09:38 Training - Downloading the training image.....Arguments: train
[2022-04-10:07:10:27:INFO] Running standalone xgboost training.
[2022-04-10:07:10:27:INFO] File size need to be processed in the node: 0.31mb. Available memory size in the node: 8153.15mb
[2022-04-10:07:10:27:INFO] Determined delimiter of CSV input is ','
[07:10:27] S3DistributionType set as FullyReplicated
[07:10:27] 2100x15 matrix with 31500 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2022-04-10:07:10:27:INFO] Determined delimiter of CSV input is ','
[07:10:27] S3DistributionType set as FullyReplicated
[07:10:27] 600x15 matrix with 9000 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[07:10:27] s

## Set up hosting for the model
### Create endpoint

입력 데이터에 대해 xgboost 모델의 학습이 완료되면 이 모델을 실시간 추론을 위한 엔드포인트로 배포합니다. Endpoint 생성은 5-10분 정도 소요됩니다.

---
### 아래 값을 변경해주세요.

모델 배포 및 호스팅에 사용되는 instance type을 설정합니다. 

표준 인스턴스 타입 중 하나인 **ml.m5.xlarge**를 사용합니다. 더 많은 인스턴스 타입은 [여기](https://aws.amazon.com/ko/sagemaker/pricing/)에서 확인할 수 있습니다.

---

In [11]:
%%time

endpoint_name='banking-fraud-'+strftime("%Y-%m-%d-%H-%M-%S", gmtime())

xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='PUT INSTANCE TYPE',           # ml.m5.xlarge
                           endpoint_name=endpoint_name)

-----!CPU times: user 97 ms, sys: 8.4 ms, total: 105 ms
Wall time: 2min 31s


## Perform Inference
### Make predictions using the endpoint

머신러닝 모델의 성능을 확인하기 위해 실제값과 예측값을 비교합니다. 추론용 데이터를 엔드포인트에 전달하고 결과를 받아옵니다. 데이터를 HTTP POST request로 보내기 위해 CSV 형태로 직렬화(serialize)하고 결과로 리턴되는 CSV를 디코딩합니다.

주의: SageMaker XGBoost에서 CSV포맷으로 추론할 때 요청 데이터는 타겟속성 컬럼을 포함하지 않습니다.

In [12]:
from sagemaker.serializers import CSVSerializer

xgb_predictor.serializer = CSVSerializer()

엔드포인트를 호출하는 함수를 생성합니다.

- 테스트 데이터셋을 반복 (Loop)
- rows 만큼 미니매치로 나누기
- 미니배치를 CSV string payloads로 변환 (타겟속성 변수 제거)
- XGBoost 엔드포인트를 호출하고 예측값 수신
- CSV 결과로 리턴된 예측값을 다시 NumPy 배열로 변환

In [13]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for i, array in enumerate(split_array):
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])
        if i % 10 == 0:
            print(i, 'out of', len(split_array), 'completed')
    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(columns_to_drop, axis=1).to_numpy())

0 out of 1 completed


In [14]:
# F1-score, accurancy, ROC 값을 확인합니다
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score # import classification metrics

print(classification_report(test_data[target], np.round(predictions)))
print("Test accuracy:", accuracy_score(test_data[target], np.round(predictions)))
print("ROC_AUC score:", roc_auc_score(test_data[target], np.round(predictions) / 1000))

              precision    recall  f1-score   support

           0       0.98      0.99      0.98       265
           1       0.91      0.86      0.88        35

    accuracy                           0.97       300
   macro avg       0.95      0.92      0.93       300
weighted avg       0.97      0.97      0.97       300

Test accuracy: 0.9733333333333334
ROC_AUC score: 0.922911051212938


예측결과와 실제값을 비교하는 Confusion matrix를 생성합니다. 알고리즘의 샘플링과정에서 랜덤요소가 반영되므로 결과의 숫자는 위 결과와 정확히 동일하지 않을 수 있습니다.

In [15]:
pd.crosstab(index=test_data['isFraud'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

predictions,0.0,1.0
actuals,,
0,262,3
1,5,30


TP, TN, FP, FN 은 다음과 같이 정의 되어 있습니다.

- TP = Truly (identified as) Positive
- TN = Truly (identified as) Negative
- FP = Falsely (identified as) Positive
- FN = Falsely (identified as) Negative

위의 테이블의 각 셀에 해당하는 것을 표시하였습니다.

| actuals\predictions | 0 | 1 |
| --- | --- | --- |
| 0 | TN | FP |
| 1 | FN | TP |


이를 바탕으로 Accuracy, Precision, Recall 을 측정할 수 있습니다.
- Accuracy = (TP + TN) / (TP + FP + FN + TP)
- Precision = TP / (TP + FP) = 670 / (670 + 29)
- Recall = TP / (TP + FN) = 670 / (670 + 157)

## Stop / Close the Endpoint

본 예제를 모두 마무리한 후 아래 셀을 실행합니다. 다음 명령은 추론 단계에서 생성한 SageMaker에서 호스팅되고 있는 엔드포인트를 제거합니다. 엔드포인트를 삭제하지 않으면 계속 사용요금이 발생할 수 있습니다.

In [16]:
# xgb_predictor.delete_endpoint()
print("Endpoint is deleted!")

Endpoint is deleted!
